In [ ]:
from dotenv import load_dotenv
load_dotenv("../.env")

True

In [ ]:
from data_retriever import data_lake

d:\Softwares\anaconda\envs\Langchain-RAG\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [ ]:
db, retriever, compression_retriever = data_lake()

2024-08-21 16:31:03.111 
  command:

    streamlit run d:\Softwares\anaconda\envs\Langchain-RAG\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
d:\Softwares\anaconda\envs\Langchain-RAG\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `CohereEmbeddings` was deprecated in LangChain 0.0.30 and will be removed in 0.3.0. An updated version of the class exists in the langchain-cohere package and should be used instead. To use it run `pip install -U langchain-cohere` and import as `from langchain_cohere import CohereEmbeddings`.
  warn_deprecated(
d:\Softwares\anaconda\envs\Langchain-RAG\lib\site-packages\pydantic\_internal\_config.py:341: UserWarning: Valid config keys have changed in V2:
* 'allow_population_by_field_name' has been renamed to 'populate_by_name'
* 'smart_union' has been removed
  warnings.warn(message, UserWarning)


Deep Lake Dataset in hub://vijayv2807/cohere-llmu-data already exists, loading from the storage


d:\Softwares\anaconda\envs\Langchain-RAG\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `CohereRerank` was deprecated in LangChain 0.0.30 and will be removed in 0.3.0. An updated version of the class exists in the langchain-cohere package and should be used instead. To use it run `pip install -U langchain-cohere` and import as `from langchain_cohere import CohereRerank`.
  warn_deprecated(


In [ ]:
db

In [ ]:
retriever

VectorStoreRetriever(tags=['DeepLake', 'CohereEmbeddings'], vectorstore=<langchain_community.vectorstores.deeplake.DeepLake object at 0x000001C6C7AD42B0>, search_type='mmr', search_kwargs={'k': 25, 'fetch_k': 8, 'distance_metric': 'cos'})

In [ ]:
compression_retriever

ContextualCompressionRetriever(base_compressor=CohereRerank(client=<cohere.client.Client object at 0x000001C6C95E7E20>, top_n=5, model='rerank-english-v3.0', cohere_api_key=None, user_agent='langchain'), base_retriever=VectorStoreRetriever(tags=['DeepLake', 'CohereEmbeddings'], vectorstore=<langchain_community.vectorstores.deeplake.DeepLake object at 0x000001C6C7AD42B0>, search_type='mmr', search_kwargs={'k': 25, 'fetch_k': 8, 'distance_metric': 'cos'}))

In [ ]:
from langchain.chains.conversational_retrieval.base import ConversationalRetrievalChain

In [ ]:
from langchain.memory import ConversationBufferWindowMemory

In [ ]:
def memory():
    memory = ConversationBufferWindowMemory(
        memory_key="chat_history", k=5, return_messages=True, output_key="answer"
    )
    return memory
memory = memory()

In [ ]:
memory

ConversationBufferWindowMemory(output_key='answer', return_messages=True, memory_key='chat_history')

In [ ]:
qa = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=compression_retriever,
    memory=memory,
    verbose=True,
    chain_type="stuff",
    return_source_documents=True,
)

NameError: name 'llm' is not defined

In [ ]:
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

NameError: name 'ChatOpenAI' is not defined

In [ ]:
from langchain_openai import ChatOpenAI

In [ ]:
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

In [ ]:
qa = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=compression_retriever,
    memory=memory,
    verbose=True,
    chain_type="stuff",
    return_source_documents=True,
)

In [ ]:
results = qa.invoke("What is Cohere mainly used for?")



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
# RESPONSE Here is a list of potential FAQs based on the provided text: - **Q: What does the Cohere Platform offer to developers and organizations?** A: The Cohere Platform offers an API that provides access to advanced Large Language Models (LLMs) without requiring machine learning expertise. It simplifies the process by handling data curation, model development, training, and serving. - **Q: What are the key language processing capabilities of the Cohere Platform?** A: The platform offers two main capabilities: text generation and text embedding. Text generation involves completing a prompt with a stream of generated text, like writing a haiku. Text embedding returns a numerical representation of t

In [ ]:
results

{'question': 'What is Cohere mainly used for?',
 'chat_history': [],
 'answer': 'Cohere is mainly used for providing access to advanced Large Language Models (LLMs) through an API, enabling developers and organizations to implement language processing capabilities such as text generation and text embedding without requiring machine learning expertise. It simplifies the process of building applications that involve natural language understanding and generation.',
 'source_documents': [Document(metadata={'title': 'Prompt Engineering Basics', 'source': 'https://cohere.com/llmu/prompt-engineering-basics', 'relevance_score': 0.9980657}, page_content="# RESPONSE Here is a list of potential FAQs based on the provided text: - **Q: What does the Cohere Platform offer to developers and organizations?** A: The Cohere Platform offers an API that provides access to advanced Large Language Models (LLMs) without requiring machine learning expertise. It simplifies the process by handling data curation

In [ ]:
# Prompt To Generate Search Query For Retriever
search_query = ChatPromptTemplate.from_messages(
    [
        MessagesPlaceholder(variable_name="chat_history"),
        ("user", "{input}"),
        (
            "user",
            "Given the above conversation, generate a search query to look up to get information relevant to the conversation",
        ),
    ]
)

# Chain that takes conversation history and returns documents.
retriever_chain = create_history_aware_retriever(
    llm=llm, retriever=compression_retriever, prompt=search_query
)

# Prompt To Get Response From LLM Based on Chat History
fetch_answer = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Answer the user's questions based on the below context:\\n\\n{context}",
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        ("user", "{input}"),
    ]
)

# Chain for passing a list of Documents to a model
document_chain = create_stuff_documents_chain(llm=llm, prompt=fetch_answer)

# Create retrieval chain that retrieves documents and then passes them on to the LLM
retrieval_chain = create_retrieval_chain(retriever_chain, document_chain)

NameError: name 'ChatPromptTemplate' is not defined

In [ ]:
import streamlit as st
from langchain.memory import ConversationBufferWindowMemory
from langchain_openai import ChatOpenAI
from langchain.chains.retrieval import create_retrieval_chain
from langchain.chains.history_aware_retriever import create_history_aware_retriever
from langchain.chains.combine_documents import create_stuff_documents_chain

from langchain.chains.conversational_retrieval.base import ConversationalRetrievalChain

from data_retriever import data_lake

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import MessagesPlaceholder

NameError: name 'ConversationalRetrievalChain' is not defined

In [ ]:
import streamlit as st
from langchain.memory import ConversationBufferWindowMemory
from langchain_openai import ChatOpenAI
from langchain.chains.retrieval import create_retrieval_chain
from langchain.chains.history_aware_retriever import create_history_aware_retriever
from langchain.chains.combine_documents import create_stuff_documents_chain

from langchain.chains.conversational_retrieval.base import ConversationalRetrievalChain

from data_retriever import data_lake

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import MessagesPlaceholder

NameError: name 'ConversationalRetrievalChain' is not defined

In [ ]:
from langchain.chains.retrieval import create_retrieval_chain

NameError: name 'ConversationalRetrievalChain' is not defined

In [ ]:
from langchain.chains.retrieval import create_retrieval_chain

In [ ]:
import streamlit as st
from langchain.memory import ConversationBufferWindowMemory
from langchain_openai import ChatOpenAI
from langchain.chains.retrieval import create_retrieval_chain
from langchain.chains.history_aware_retriever import create_history_aware_retriever
from langchain.chains.combine_documents import create_stuff_documents_chain

from langchain.chains.conversational_retrieval.base import ConversationalRetrievalChain

from data_retriever import data_lake

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import MessagesPlaceholder

In [ ]:
def memory():
    memory = ConversationBufferWindowMemory(
        memory_key="chat_history", k=5, return_messages=True, output_key="answer"
    )
    return memory
memory = memory()


llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)


# Prompt To Generate Search Query For Retriever
search_query = ChatPromptTemplate.from_messages(
    [
        MessagesPlaceholder(variable_name="chat_history"),
        ("user", "{input}"),
        (
            "user",
            "Given the above conversation, generate a search query to look up to get information relevant to the conversation",
        ),
    ]
)

# Chain that takes conversation history and returns documents.
retriever_chain = create_history_aware_retriever(
    llm=llm, retriever=compression_retriever, prompt=search_query
)

# Prompt To Get Response From LLM Based on Chat History
fetch_answer = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Answer the user's questions based on the below context:\\n\\n{context}",
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        ("user", "{input}"),
    ]
)

# Chain for passing a list of Documents to a model
document_chain = create_stuff_documents_chain(llm=llm, prompt=fetch_answer)

# Create retrieval chain that retrieves documents and then passes them on to the LLM
retrieval_chain = create_retrieval_chain(retriever_chain, document_chain)

In [ ]:
from langchain_core.messages import HumanMessage, AIMessage

In [ ]:
response = retrieval_chain.invoke({
"chat_history": memory.memory_variables(),
"input":"What is Cohere mainly used for?"
})

TypeError: 'list' object is not callable

In [ ]:
memory

ConversationBufferWindowMemory(output_key='answer', return_messages=True, memory_key='chat_history')

In [ ]:
con_memory = memory()

TypeError: 'ConversationBufferWindowMemory' object is not callable

In [ ]:
def memory():
    memory = ConversationBufferWindowMemory(
        memory_key="chat_history", k=5, return_messages=True, output_key="answer"
    )
    return memory

In [ ]:
con_memory = memory()

In [ ]:
con_memory

ConversationBufferWindowMemory(output_key='answer', return_messages=True, memory_key='chat_history')

In [ ]:
response = retrieval_chain.invoke({
"chat_history": con_memory.load_memory_variables({}),
"input":"What is Cohere mainly used for?"
})

ValueError: variable chat_history should be a list of base messages, got {'chat_history': []} of type <class 'dict'>

In [ ]:
response = retrieval_chain.invoke({
"chat_history": con_memory.memory_variables(),
"input":"What is Cohere mainly used for?"
})

TypeError: 'list' object is not callable

In [ ]:
response = retrieval_chain.invoke({
"chat_history": con_memory.memory_variables,
"input":"What is Cohere mainly used for?"
})

In [ ]:
response

{'chat_history': ['chat_history'],
 'input': 'What is Cohere mainly used for?',
 'context': [Document(metadata={'title': 'Introduction to Cohere on Amazon Bedrock', 'source': 'https://cohere.com/llmu/co-aws-bedrock', 'relevance_score': 0.99497885}, page_content='Note: the list of features will continue to evolve and expand. To keep up with the latest, visit the Bedrock website.\nCohere Models on Amazon Bedrock\nList of Models\nThe most up-to-date list of available Cohere’s models is on the Bedrock website, but here is the list at the time of writing:\nGenerative models:\nCommand R+\nCommand R\nCommand\nCommand Light\nEmbeddings models:\nEmbed (English)\nEmbed (Multilingual)\nPricing Mechanism\nThe pricing for Cohere models on Bedrock at the time of writing falls under two types:\nOn-demand and batch\nThis applies to default Cohere models on Bedrock. Pricing is based on tokens and broken down into:\nPrice per 1,000 input tokens\nPrice per 1,000 output tokens\nCustomization (fine-tuning)